In [1]:
from tqdm.notebook import tqdm
import math
import gym
import torch
import torch.optim as optim 
from torch.utils.tensorboard import SummaryWriter
from collections import deque

from active_rl.networks.dqn_atari import MC_DQN
from active_rl.utils.memory import BALDReplayMemoryForMcDropout
from active_rl.utils.optimization import AMN_optimization
from active_rl.environments.atari_wrappers import make_atari, wrap_deepmind
from active_rl.utils.atari_utils import fp, evaluate, ActionSelector

In [2]:
env_name = 'Breakout'
env_raw = make_atari('{}NoFrameskip-v4'.format(env_name))
env = wrap_deepmind(env_raw, frame_stack=False, episode_life=True, clip_rewards=True)
c,h,w = c,h,w = fp(env.reset()).shape
n_actions = env.action_space.n

In [3]:
BATCH_SIZE = 64
LR = 0.0000625
GAMMA = 0.99
EPS_START = 1.
EPS_END = 0.01
EPS_DECAY = 1000000 
NUM_STEPS = 20000000
LABELLED_MEMORY_CAPACITY = 10000
UNLABELLED_MEMORY_CAPACITY = 10000
BATCH_LABEL_SIZE=1000
INITIAL_STEPS=1000
NUM_SAMPLES=20
TRAINING_ITERATIONS= int(10 * LABELLED_MEMORY_CAPACITY /  BATCH_SIZE)

NAME = f"AMN_mc_Bald_eps{EPS_END}_BatchLabelSize{BATCH_LABEL_SIZE}_UnlabelledCapacity{UNLABELLED_MEMORY_CAPACITY}_EPS{EPS_END}_Env{env_name}"

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # if gpu is to be used
AMN_net = MC_DQN(n_actions).to(device)
expert_net = torch.load("models/expert_mc_Breakout_step19000000").to(device)
AMN_net.apply(AMN_net.init_weights)
expert_net.eval()
optimizer = optim.Adam(AMN_net.parameters(), lr=LR, eps=1.5e-4)

In [5]:
memory = BALDReplayMemoryForMcDropout(UNLABELLED_MEMORY_CAPACITY, LABELLED_MEMORY_CAPACITY, BATCH_LABEL_SIZE, [5,h,w], n_actions, AMN_net, NUM_SAMPLES, device=device)
action_selector = ActionSelector(EPS_START, EPS_END, AMN_net, EPS_DECAY, n_actions, device)

In [6]:
steps_done = 0
writer = SummaryWriter(f'runs/{NAME}')

In [7]:
q = deque(maxlen=5)
done=True

In [ ]:
progressive = tqdm(range(NUM_STEPS), total=NUM_STEPS, ncols=400, leave=False, unit='b')
for step in progressive:
  if done:
    env.reset()
    sum_reward = 0
    img, _, _, _ = env.step(1) # BREAKOUT specific !!!
    for i in range(10): # no-op
      n_frame, _, _, _ = env.step(0)
      n_frame = fp(n_frame)
      q.append(n_frame)
        
  # Select and perform an action
  state = torch.cat(list(q))[1:].unsqueeze(0)
  action, eps = action_selector.select_action(state)
  n_frame, reward, done, info = env.step(action)
  n_frame = fp(n_frame)

  # 5 frame as memory
  q.append(n_frame)
  memory.push(torch.cat(list(q)).unsqueeze(0), action, reward, done) # here the n_frame means next frame from the previous time step

  # Perform one step of the optimization (on the target network)
  if step % UNLABELLED_MEMORY_CAPACITY == 0 and step > INITIAL_STEPS:
    memory.label_sample()
    loss = 0
      
    for _ in range(TRAINING_ITERATIONS):
      loss += AMN_optimization(AMN_net, expert_net, optimizer, memory, batch_size=BATCH_SIZE, device=device)
        
    loss /= TRAINING_ITERATIONS
    writer.add_scalar('Performance/loss', loss, step)
      
    evaluated_reward = evaluate(step, AMN_net, device, env_raw, n_actions, eps=0.01, num_episode=20)
    writer.add_scalar('Performance/reward', evaluated_reward, step)
    writer.add_scalar('Performance/reward_labels', evaluated_reward, BATCH_LABEL_SIZE * step / UNLABELLED_MEMORY_CAPACITY)